In [ ]:
!pip install gradio --quiet

In [ ]:
!pip install ultralytics --quiet

In [3]:
import gradio as gr
from ultralytics import YOLO
import cv2
from PIL import Image
import numpy as np

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Load trained model
model = YOLO('/content/drive/MyDrive/Flash Detection Project/Model_Files/Best_Model1.pt')
print("Yolov8n Loaded Successfully")

Yolov8n Loaded Successfully


In [12]:
def detect_flash_gradio(image: Image.Image):
    # Convert PIL image to OpenCV format
    image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Run detection
    results = model(image_cv)

    # Extract detection info
    detections = results[0].boxes.data.tolist()
    num_detections = len(detections)

    message = ""
    if num_detections > 0:
        message += f"Flash(es) Detected: {num_detections} \n"
        for i, det in enumerate(detections):
            x1, y1, x2, y2, conf, cls = det
            message += f"  Flash {i+1}: Confidence={conf:.2f}, Position=[{int(x1)},{int(y1)},{int(x2)},{int(y2)}]\n"
    else:
        message = "❌ No Flash Detected."

    # Plot result with bounding boxes
    res_plotted = results[0].plot()
    res_rgb = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)
    output_image = Image.fromarray(res_rgb)

    return output_image, message

In [13]:
# Gradio interface
demo = gr.Interface(
    fn=detect_flash_gradio,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=[
        gr.Image(type="pil", label="Detected Image"),
        gr.Textbox(label="Detection Info")
    ],
    title="Flash DetectioN",
    description="Upload an image to detect flash"
)

In [14]:
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://743576cdb2ca8301ff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
